In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [2]:
crime = pd.read_csv('nycrime.csv')
crime

,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NEW YORK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Offenses Known to Law Enforcement,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"by City, 2013",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,City,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
4,Adams Village,"1,861",0,0,NaN,0,0,0,12,2,10,0,0
5,Addison Town and Village,"2,577",3,0,NaN,0,0,3,24,3,20,1,0
6,Akron Village,"2,846",3,0,NaN,0,0,3,16,1,15,0,0
7,Albany,"97,956",791,8,NaN,30,227,526,"4,090",705,"3,243",142,NaN
8,Albion Village,"6,388",23,0,NaN,3,4,16,223,53,165,5,NaN
9,Alfred Village,"4,089",5,0,NaN,0,3,2,46,10,36,0,NaN


In [3]:
column = pd.Series(crime.iloc[3]).values

In [4]:
crime = crime.drop(crime.index[[0,1,2,3,352,353,354]])
crime.columns = column
crime.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
4,Adams Village,"1,861",0,0,NaN,0,0,0,12,2,10,0,0
5,Addison Town and Village,"2,577",3,0,NaN,0,0,3,24,3,20,1,0
6,Akron Village,"2,846",3,0,NaN,0,0,3,16,1,15,0,0
7,Albany,"97,956",791,8,NaN,30,227,526,"4,090",705,"3,243",142,NaN
8,Albion Village,"6,388",23,0,NaN,3,4,16,223,53,165,5,NaN


In [5]:
crime = crime.reset_index(drop=True)
feature = ['Population','Murder and\nnonnegligent\nmanslaughter','Robbery']
crime[feature].head()

,Population,Murder and nonnegligent manslaughter,Robbery
0,"1,861",0,0
1,"2,577",0,0
2,"2,846",0,0
3,"97,956",8,227
4,"6,388",0,4


In [6]:
features = pd.DataFrame()

In [7]:
#Target
features['Target'] = crime['Violent\ncrime'].str.replace(',','')
for i in range(len(crime['Violent\ncrime'])):
    features['Target'][i] = int(features['Target'][i])

In [8]:
#Population
features['Population'] = crime['Population'].str.replace(',','')
for i in range(len(crime['Population'])):
    features['Population'][i] = int(features['Population'][i])

In [9]:
#Population squared
features['Pop Squared'] = features['Population']
for i in range(len(features['Population'])):
    features['Pop Squared'][i] = int(features['Pop Squared'][i])*int(features['Pop Squared'][i])

In [10]:
#Murder
features['Murder'] = crime['Murder and\nnonnegligent\nmanslaughter'].str.replace(',','')
for i in range(len(crime['Murder and\nnonnegligent\nmanslaughter'])):
    features['Murder'][i] = int(features['Murder'][i])
    if features['Murder'][i] == 0:
        features['Murder'][i] = 0
    else:
        features['Murder'][i] = 1

In [11]:
#Robbery
features['Robbery'] = crime['Robbery'].str.replace(',','')
for i in range(len(crime['Robbery'])):
    features['Robbery'][i] = int(features['Robbery'][i])
    if features['Robbery'][i] == 0:
        features['Robbery'][i] = 0
    else:
        features['Robbery'][i] = 1

In [12]:
features

,Target,Population,Pop Squared,Murder,Robbery
0,0,1861,3463321,0,0
1,3,2577,6640929,0,0
2,3,2846,8099716,0,0
3,791,97956,9595377936,1,1
4,23,6388,40806544,0,1
5,5,4089,16719921,0,1
6,3,1781,3171961,0,0
7,107,118296,13993943616,1,1
8,9,9519,90611361,0,1
9,30,18182,330585124,0,1
